In [179]:
import pandas as pd
import numpy as np
import pandas
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [180]:
file_path = "Unclean_data.csv"  # Path to your CSV file
data = pd.read_csv(file_path)

In [181]:
print("Data loaded successfully.")
print(f"Data Shape: {data.shape}")
print(data.head())


Data loaded successfully.
Data Shape: (1025, 8)
        Location      Type     Category               Description  Price  \
0     El Manar 1  Location  App. 3 pièc  Mignon appartement meubl  1 500   
1    El Menzah 9  Location  App. 3 pièc  Magnifique appartement s  1 350   
2        Gammart  Location      Maisons   Triplex s4 avec piscine  7 500   
3  Marsa Erriadh  Location  App. 4 pièc      Appart s3 samar à la  1 250   
4        Gammart  Location       Duplex  Duplex meuble s3 a gamma  4 500   

         Date  Rooms Surface  
0  19/12/2024    2.0     NaN  
1  19/12/2024    2.0     NaN  
2  19/12/2024    4.0     NaN  
3  19/12/2024    3.0  130 m²  
4  19/12/2024    3.0     NaN  


In [182]:
# Check for missing values
print("\nMissing Values:")
print(data.isnull().sum())


Missing Values:
Location         0
Type             0
Category         0
Description      0
Price            0
Date             0
Rooms          537
Surface        315
dtype: int64


In [183]:
# Display the number of duplicate rows before cleaning
print(f"Number of duplicate rows before cleaning: {data.duplicated().sum()}")


Number of duplicate rows before cleaning: 777


In [184]:
# Quick overview of statistical data
print("\nDescriptive Statistics:")
print(data.describe(include="all"))


Descriptive Statistics:
       Location      Type Category               Description  Price  \
count      1025      1025     1025                      1025   1025   
unique       71         5       15                       227    119   
top     Gammart  Location  Maisons  Villa avec piscine à gam  6 500   
freq        112       655      355                        32     41   
mean        NaN       NaN      NaN                       NaN    NaN   
std         NaN       NaN      NaN                       NaN    NaN   
min         NaN       NaN      NaN                       NaN    NaN   
25%         NaN       NaN      NaN                       NaN    NaN   
50%         NaN       NaN      NaN                       NaN    NaN   
75%         NaN       NaN      NaN                       NaN    NaN   
max         NaN       NaN      NaN                       NaN    NaN   

              Date       Rooms Surface  
count         1025  488.000000     710  
unique           1         NaN      77  

In [185]:
# Remove rows where 'Rooms' is missing or has 'N/A'
data['Rooms'] = data['Rooms'].replace('N/A', np.nan)  # Replace 'N/A' with NaN
data = data.dropna(subset=['Rooms'])  # Drop rows with NaN in 'Rooms'

In [186]:
# Convert 'Rooms' to integer after dropping NaN values
data['Rooms'] = data['Rooms'].astype(int)

In [187]:
print(f"Data Shape after removing rows with null 'Rooms': {data.shape}")
print(data.head())

Data Shape after removing rows with null 'Rooms': (488, 8)
        Location      Type     Category               Description  Price  \
0     El Manar 1  Location  App. 3 pièc  Mignon appartement meubl  1 500   
1    El Menzah 9  Location  App. 3 pièc  Magnifique appartement s  1 350   
2        Gammart  Location      Maisons   Triplex s4 avec piscine  7 500   
3  Marsa Erriadh  Location  App. 4 pièc      Appart s3 samar à la  1 250   
4        Gammart  Location       Duplex  Duplex meuble s3 a gamma  4 500   

         Date  Rooms Surface  
0  19/12/2024      2     NaN  
1  19/12/2024      2     NaN  
2  19/12/2024      4     NaN  
3  19/12/2024      3  130 m²  
4  19/12/2024      3     NaN  


In [188]:
# Check for missing values
print("\nMissing Values:")
print(data.isnull().sum())


Missing Values:
Location         0
Type             0
Category         0
Description      0
Price            0
Date             0
Rooms            0
Surface        141
dtype: int64


In [189]:
# Convert Surface to numeric and handle missing values
data['Surface'] = (
    data['Surface']
    .astype(str)  # Convert all values to string
    .str.replace(' m²', '', regex=False)
    .str.replace(',', '', regex=False)
    .replace('N/A', np.nan)
    .astype(float)  # Convert back to float
)

# Ensure Rooms column is numeric
data['Rooms'] = data['Rooms'].replace('N/A', np.nan).astype(float)

# Calculate mean Surface for each Rooms count, ignoring nulls
mean_surface_per_room = data.groupby('Rooms')['Surface'].mean()

# Fill missing Surface values with the mean based on Rooms
def fill_surface(row):
    if pd.isnull(row['Surface']):
        return mean_surface_per_room.get(row['Rooms'], np.nan)
    return row['Surface']

data['Surface'] = data.apply(fill_surface, axis=1)

# Reset index after updating
data = data.reset_index(drop=True)
# Convert 'Rooms' to integer after dropping NaN values
data['Rooms'] = data['Rooms'].astype(int)

print(f"Number of missing Surface values after filling: {data['Surface'].isnull().sum()}")





Number of missing Surface values after filling: 7


In [190]:
# Check for missing values
print("\nMissing Values:")
print(data.isnull().sum())


Missing Values:
Location       0
Type           0
Category       0
Description    0
Price          0
Date           0
Rooms          0
Surface        7
dtype: int64


In [191]:
# Display rows with null Surface values
null_surface_rows = data[data['Surface'].isnull()]
print("\nRows with null Surface values:")
print(null_surface_rows)

# Show the count of rows with null Surface
print(f"\nNumber of rows with null Surface: {len(null_surface_rows)}")

# Drop rows with null Surface values
data = data.dropna(subset=['Surface'])

# Reset the index after dropping rows
data = data.reset_index(drop=True)

# Verify the result
print("\nData after dropping rows with null Surface:")
print(data.head())
print(f"Number of rows after dropping null Surface values: {len(data)}")



Rows with null Surface values:
         Location      Type Category               Description      Price  \
152    El Manar 1     Vente  Maisons  Grande villa s6 avec gra  1 300 000   
162  Berge Du Lac  Location  Maisons    Villa haut standing s6      9 500   
166    El Manar 1     Vente  Maisons  Grande villa s6 avec gra  1 300 000   
185  Berge Du Lac  Location  Maisons    Villa haut standing s6      9 500   
189    El Manar 1     Vente  Maisons  Grande villa s6 avec gra  1 300 000   
213  Berge Du Lac  Location  Maisons    Villa haut standing s6      9 500   
217    El Manar 1     Vente  Maisons  Grande villa s6 avec gra  1 300 000   

           Date  Rooms  Surface  
152  19/12/2024      6      NaN  
162  19/12/2024      6      NaN  
166  19/12/2024      6      NaN  
185  19/12/2024      6      NaN  
189  19/12/2024      6      NaN  
213  19/12/2024      6      NaN  
217  19/12/2024      6      NaN  

Number of rows with null Surface: 7

Data after dropping rows with null Surfac

In [192]:
# Remove duplicate rows
data = data.drop_duplicates()

# Reset the index after removing duplicates
data = data.reset_index(drop=True)

# Display the number of duplicate rows after cleaning
print(f"Number of duplicate rows after cleaning: {data.duplicated().sum()}")

# Verify the result
print("\nData after removing duplicates:")
print(data.head())

Number of duplicate rows after cleaning: 0

Data after removing duplicates:
        Location      Type     Category               Description  Price  \
0     El Manar 1  Location  App. 3 pièc  Mignon appartement meubl  1 500   
1    El Menzah 9  Location  App. 3 pièc  Magnifique appartement s  1 350   
2        Gammart  Location      Maisons   Triplex s4 avec piscine  7 500   
3  Marsa Erriadh  Location  App. 4 pièc      Appart s3 samar à la  1 250   
4        Gammart  Location       Duplex  Duplex meuble s3 a gamma  4 500   

         Date  Rooms     Surface  
0  19/12/2024      2  113.408163  
1  19/12/2024      2  113.408163  
2  19/12/2024      4  159.090909  
3  19/12/2024      3  130.000000  
4  19/12/2024      3  197.720588  


In [193]:
# Filter the DataFrame to keep only rows with specified categories
valid_categories = ['Location', 'Location vacance', 'Vente']
data = data[data['Type'].isin(valid_categories)]  # Filter based on the 'Type' column

# Reset the index after filtering
data = data.reset_index(drop=True)

# Verify the result
print(f"Number of rows after filtering: {len(data)}")
print(data.head())


Number of rows after filtering: 115
        Location      Type     Category               Description  Price  \
0     El Manar 1  Location  App. 3 pièc  Mignon appartement meubl  1 500   
1    El Menzah 9  Location  App. 3 pièc  Magnifique appartement s  1 350   
2        Gammart  Location      Maisons   Triplex s4 avec piscine  7 500   
3  Marsa Erriadh  Location  App. 4 pièc      Appart s3 samar à la  1 250   
4        Gammart  Location       Duplex  Duplex meuble s3 a gamma  4 500   

         Date  Rooms     Surface  
0  19/12/2024      2  113.408163  
1  19/12/2024      2  113.408163  
2  19/12/2024      4  159.090909  
3  19/12/2024      3  130.000000  
4  19/12/2024      3  197.720588  


In [194]:
# Save the cleaned data to a new CSV
cleaned_file_path = "Clean_data.csv"
data.to_csv(cleaned_file_path, index=False)
print(f"Cleaned data saved to {cleaned_file_path}")


Cleaned data saved to Clean_data.csv
